Lo que tiene que hacer:
- Descartar las películas que no tienen “overview”. 
- Descartar las películas cuyo idioma original tenga más de 2 caracteres.
- Tomar las 100 palabras más comunes de todos los overviews combinados.
- Para generar el ”Overview” de cada película para el archivo de salida se deberán descartar las palabras que se encuentren dentro de las 100 más comunes de todos los overviews y se deberá generar un string con 3 palabras tomadas al azar y ordenadas al azar (ver random.sample()) de ese resultado.
- Se utilizarán como datos de las tarjetas: “Genre”, “Original_Language”, “Vote_Average”, “Release_Date” y 3 palabras aleatorias del overview separadas por “;”. Como dato a adivinar se utilizará “Title”. Descartar el resto de las columnas.
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): “Genre”, “Original_Language”, “Release_Date”,“Vote_Average”, “Overview” y “Title”.

## Imports Necesarios

In [1]:
import os
import pandas as pd
import string
import collections
import random

## Funciones
limpiar: Elimina los caracteres especiales del string enviado.  
filtrar_cien: Se queda con a lo sumo tres palabras del string que no estén en las cien palabras más usadas

In [2]:
def limpiar(cadena):
    """Elimina una serie de caracteres de la cadena ingresada."""
    for c in string.punctuation:
        cadena = cadena.replace(c, "")
    return cadena

def filtrar_cien(cad, cien):
    """Elimina las palabras que estén en los cien y toma 3"""
    cad = cad.split()
    cad = [c for c in cad if not c in cien]
    if len(cad) >= 3:
        cad = str(random.sample(cad,3))
    else:
        cad = str(random.sample(cad,len(cad)))
    return cad.replace("[","").replace("]","").replace(",",";").replace("'","").replace('"',"")

Leo el dataset mediante pandas.

In [3]:
path_dataset = os.path.join(os.getcwd(),"mymoviedb.csv")
try:
    data_set = pd.read_csv(path_dataset, encoding='utf-8', lineterminator='\n')
except FileNotFoundError:
    print("No se encontró el dataset mymoviedb.csv")
else:
    print(data_set)

     Release_Date                                 Title  \
0      2021-12-15               Spider-Man: No Way Home   
1      2022-03-01                            The Batman   
2      2022-02-25                               No Exit   
3      2021-11-24                               Encanto   
4      2021-12-22                        The King's Man   
...           ...                                   ...   
9822   1973-10-15                              Badlands   
9823   2020-10-01                      Violent Delights   
9824   2016-05-06                          The Offering   
9825   2021-03-31  The United States vs. Billie Holiday   
9826   1984-09-23                               Threads   

                                               Overview  Popularity  \
0     Peter Parker is unmasked and no longer able to...    5083.954   
1     In his second year of fighting crime, Batman u...    3827.658   
2     Stranded at a rest stop in the mountains durin...    2618.087   
3     T

Descarto las filas que no tengan overview.  
Después elimino los caracteres especiales de los overviews

In [4]:
data_set = data_set[data_set['Overview'] != ""]
data_set['Overview'] = list(map(limpiar, data_set['Overview']))
print(data_set['Overview'])

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
0       Peter Parker is unmasked and no longer able to...
1       In his second year of fighting crime Batman un...
2       Stranded at a rest stop in the mountains durin...
3       The tale of an extraordinary family the Madrig...
4       As a collection of historys worst tyrants and ...
                              ...                        
9822    A dramatization of the StarkweatherFugate kill...
9823    A female vampire falls in love with a man she ...
9824    When young and successful reporter Jamie finds...
9825    Billie Holiday spent much of her career being ...
9826    Documentary style account of a nuclear holocau...
Name: Overview, Length: 9827, dtype: object


Obtengo en la variable cien las cien palabras más repetidas en la suma de los overviews.  
Luego para cada overview me quedo con tres palabras que no se encuentren en la variable cien.

In [5]:
cien = []
for dat in data_set['Overview']:
    cien += dat.split()
cien = sorted(dict(collections.Counter(cien)).items(), key=lambda x: x[1])[-100:]
cien = [x[0] for x in cien]

data_set['Overview'] = list(map(lambda cad: filtrar_cien(cad, cien), data_set['Overview']))
print(data_set['Overview'])

0                  Strange; separate; Doctor
1                Batman; corruption; Riddler
2             discovers; mountains; building
3                       magical; last; child
4                    tyrants; criminal; race
                        ...                 
9822    spree; boyfriend; StarkweatherFugate
9823                  pregnant; wife; female
9824               husband; entity; reporter
9825                 Holiday; ballad; effort
9826                 nuclear; run; holocaust
Name: Overview, Length: 9827, dtype: object


Elimino las columnas que no sirven: Popularity, Vote_Count, Poster_Url.  
Reordeno las columnas restantes a como se piden: Genre, Original_Language, Release_Date, Vote_Average, Overview, Title.

In [6]:
columnas = ["Popularity", "Vote_Count", "Poster_Url"]
data_set = data_set.drop(columnas, axis='columns')

columnas = ["Genre","Original_Language","Release_Date","Vote_Average","Overview","Title"]
data_set = data_set.reindex(columns=columnas)
print(data_set)

                                   Genre Original_Language Release_Date  \
0     Action, Adventure, Science Fiction                en   2021-12-15   
1               Crime, Mystery, Thriller                en   2022-03-01   
2                               Thriller                en   2022-02-25   
3     Animation, Comedy, Family, Fantasy                en   2021-11-24   
4       Action, Adventure, Thriller, War                en   2021-12-22   
...                                  ...               ...          ...   
9822                        Drama, Crime                en   1973-10-15   
9823                              Horror                es   2020-10-01   
9824           Mystery, Thriller, Horror                en   2016-05-06   
9825               Music, Drama, History                en   2021-03-31   
9826         War, Drama, Science Fiction                en   1984-09-23   

      Vote_Average                              Overview  \
0              8.3             Strange;

Por ultimo, guardo los datos en dataset_peliculas.csv

In [7]:
path_dataset = os.path.join(os.path.realpath(".."), "dataset_peliculas.csv")
data_set.to_csv(path_dataset, index=False)